# Work with https://midkam.ru/ database

In [20]:
import pandas as pd

## Read csv files

In [21]:
data = {
    'kamaz': pd.read_csv('midkam/KamazPrice.csv'),
    'gaz': pd.read_csv('midkam/GazPrice.csv'),
    'maz': pd.read_csv('midkam/MazPrice.csv')
}

## Drop columns

In [22]:
data['kamaz'] = data['kamaz'].drop(
    columns=['Код заказа', 'Количество заказано', 'Сумма заказа руб.', 'Вес заказа кг.', 'Вес кг.', 'Ед.изм.'])
data['gaz'] = data['gaz'].drop(
    columns=['Количество', 'Сумма, руб', 'Ед. изм.'])
data['maz'] = data['maz'].drop(
    columns=['Код заказа', 'Количество заказано', "Вес кг.", "Сумма заказа руб.", "Вес заказа кг.", "Ед.изм."])

## Rename columns and change product price

In [23]:
for title in data:
    data[title] = data[title].rename(columns={
        'Номер по каталогу': 'vendor_code',
        'Цена руб': 'price',
        'Наименование': 'title'
    })
    data[title]['price'] = data[title]['price'].replace(',', '', regex=True)
    data[title] = data[title].astype({'price': float})
    data[title]['price'] *= 1.2

## Set product category

In [24]:
data['kamaz']['category'] = 1
data['gaz']['category'] = 6
data['maz']['category'] = 2

## Joining tables

In [25]:
midkam_data = pd.concat(data.values())
midkam_data

,vendor_code,title,price,category
0,8.3-20,Автокамера 8.3-20,1440.0,1
1,12.00-16,Автокамера 12.00-16 / г.Барнаул,1500.0,1
2,12.00-24,Автокамера 12.00-24 / г. Нижнекамск,5812.8,1
3,-,Автокамера 1200x500x508 (Барнаул),7128.0,1
4,13.6-38,Автокамера 13.6-38 (Барнаул),3120.0,1
...,...,...,...,...
5718,40001,Ящик инструментальный МАЗ (пластик),6234.0,2
5719,NaN,Ящик тара до 500кг дл-110мм шир-75мм выс-78мм,1320.0,2
5720,5336-,Ящик-тара для стекол МАЗ,2640.0,2
5721,6430-,Ящик-тара для стекол МАЗ 6430,3300.0,2


# Work with DR database from their website

In [26]:
dt_data = pd.read_csv('datasets/dt_data_1.csv', sep=';')

dt_data = dt_data.drop(columns=['id'])

dt_data = dt_data.rename(
    columns={
        'Артикул': 'vendor_code',
        'Имя': 'title',
        'Краткое описание': 'short_description',
        'Описание': 'description',
        'Категории': 'category',
        'Базовая цена': 'price',
        'Изображения': 'image_original'
    })

dt_data['image_128'] = None
dt_data['image_432'] = None

for index in range(dt_data.shape[0]):
    img_url = dt_data.loc[index, 'image_original']
    if type(img_url) != float:
        img_name = img_url.split(', ')[0].split('/')[-1]

        dt_data.loc[index, 'image_original'] = 'original/' + img_name
        dt_data.loc[index, 'image_432'] = '432/432__' + img_name
        dt_data.loc[index, 'image_128'] = '128/128__' + img_name

dt_data['description'] = dt_data['description'].fillna(dt_data['title'])
dt_data['short_description'] = dt_data['short_description'].fillna(dt_data['description'])

dt_data = dt_data.fillna(0)

dt_data

,vendor_code,title,short_description,description,price,category,image_original,image_128,image_432
0,55111-2918068,Башмак балансира с алюминевой втулкой,Торговая марка: КАМАЗАртикул: 5511-2918068Габа...,Башмак балансира с алюминиевой втулкойАртикул:...,4050.0,1,original/IMG_20190703_101637.jpg,128/128__IMG_20190703_101637.jpg,432/432__IMG_20190703_101637.jpg
1,55111-2918070-1,Башмак балансира с бронзовой втулкой,Торговая марка: КАМАЗАртикул: 5511-2918068Габа...,Башмак балансира с бронзовой втулкой КамАЗАрти...,5150.0,1,original/IMG_20190703_101637.jpg,128/128__IMG_20190703_101637.jpg,432/432__IMG_20190703_101637.jpg
2,55111-2918070-1-1,Башмак балансира с фторопластовой втулкой,Торговая марка: КАМАЗАртикул: 5511-2918068Габа...,Башмак балансира с фторопластовой втулкойАртик...,4350.0,1,original/IMG_20190703_101637.jpg,128/128__IMG_20190703_101637.jpg,432/432__IMG_20190703_101637.jpg
3,740.1003010-20,Головка блока цилиндров Камаз в сборе,Производитель: КАМАЗАртикул: 740-1003010-20Шир...,Головка блока цилиндров Камаз в сборе 740.1003...,3000.0,1,original/IMG_20190619_134642.jpg,128/128__IMG_20190619_134642.jpg,432/432__IMG_20190619_134642.jpg
4,740.30-1003010,"Головка блока цилиндров Камаз ЕВРО-1, 2, 3 в с...",Производитель: КАМАЗАртикул: 740.30-1003010Шир...,Головка блока КамАЗ ЕВРО(740.30-1003010)Головк...,5000.0,1,original/IMG_20190520_103858-scaled.jpg,128/128__IMG_20190520_103858-scaled.jpg,432/432__IMG_20190520_103858-scaled.jpg
...,...,...,...,...,...,...,...,...,...
149,C700 VW 717-150,ГУР (Механизм рулевой) КАМАЗ-65221 C700 VW 717...,"Применяется на а/м КАМАЗ: 6522, 65224, 65225, ...","Применяется на а/м КАМАЗ: 6522, 65224, 65225, ...",44000.0,1,original/IMG_20190315_131613-scaled.jpg,128/128__IMG_20190315_131613-scaled.jpg,432/432__IMG_20190315_131613-scaled.jpg
150,C700 VW 717-078,ГУР (Механизм рулевой) КАМАЗ-6520 C700 VW 717-...,"Механизм рулевой ГУР Камаз, УРАЛ C700 V 717-07...","Механизм рулевой ГУР Камаз, УРАЛ C700 V 717-07...",45000.0,1,original/IMG_1434-06-10-21-09-03.jpg,128/128__IMG_1434-06-10-21-09-03.jpg,432/432__IMG_1434-06-10-21-09-03.jpg
151,6.1-7.01,"Турбокомпрессор ПАЗ дв. 245.7,245.9 Евро-2 (6....",Регулируемый турбокомпрессор с перепуском газа...,Регулируемый турбокомпрессор с перепуском газа...,13500.0,5,original/запчасть-на-фотосессии.jpg,128/128__запчасть-на-фотосессии.jpg,432/432__запчасть-на-фотосессии.jpg
152,6.5.1-17.01.03,"Турбокомпрессор ГАЗ, ММЗ Д-245.7Е3 ТКР 6.5.1-1...",Турбокомпрессор предназначен для комплектации ...,Турбокомпрессор предназначен для комплектации ...,13500.0,6,original/запчасть-на-фотосессии.jpg,128/128__запчасть-на-фотосессии.jpg,432/432__запчасть-на-фотосессии.jpg


## Drop duplicates and NaN

In [27]:
df = pd.concat([dt_data, midkam_data])

df = df.drop_duplicates(subset=['vendor_code'])
df = df.drop_duplicates(subset=['title'])
df = df.dropna(subset=['vendor_code', 'title'])

df.reset_index(drop=True)

,vendor_code,title,short_description,description,price,category,image_original,image_128,image_432
0,55111-2918068,Башмак балансира с алюминевой втулкой,Торговая марка: КАМАЗАртикул: 5511-2918068Габа...,Башмак балансира с алюминиевой втулкойАртикул:...,4050.0,1,original/IMG_20190703_101637.jpg,128/128__IMG_20190703_101637.jpg,432/432__IMG_20190703_101637.jpg
1,55111-2918070-1,Башмак балансира с бронзовой втулкой,Торговая марка: КАМАЗАртикул: 5511-2918068Габа...,Башмак балансира с бронзовой втулкой КамАЗАрти...,5150.0,1,original/IMG_20190703_101637.jpg,128/128__IMG_20190703_101637.jpg,432/432__IMG_20190703_101637.jpg
2,55111-2918070-1-1,Башмак балансира с фторопластовой втулкой,Торговая марка: КАМАЗАртикул: 5511-2918068Габа...,Башмак балансира с фторопластовой втулкойАртик...,4350.0,1,original/IMG_20190703_101637.jpg,128/128__IMG_20190703_101637.jpg,432/432__IMG_20190703_101637.jpg
3,740.1003010-20,Головка блока цилиндров Камаз в сборе,Производитель: КАМАЗАртикул: 740-1003010-20Шир...,Головка блока цилиндров Камаз в сборе 740.1003...,3000.0,1,original/IMG_20190619_134642.jpg,128/128__IMG_20190619_134642.jpg,432/432__IMG_20190619_134642.jpg
4,740.30-1003010,"Головка блока цилиндров Камаз ЕВРО-1, 2, 3 в с...",Производитель: КАМАЗАртикул: 740.30-1003010Шир...,Головка блока КамАЗ ЕВРО(740.30-1003010)Головк...,5000.0,1,original/IMG_20190520_103858-scaled.jpg,128/128__IMG_20190520_103858-scaled.jpg,432/432__IMG_20190520_103858-scaled.jpg
...,...,...,...,...,...,...,...,...,...
22000,840-1117030,Элемент ФТОТ 840 (Евро-3) (нитка) СЕДАН,NaN,NaN,231.6,2,NaN,NaN,NaN
22001,2501-3708200-21,Якорь (для старт.2501.3708000-21) 10зуб. н/о П...,NaN,NaN,9000.0,2,NaN,NaN,NaN
22002,2501-3708200-20(01),Якорь стартера МАЗ (11 зубьев) старого образца...,NaN,NaN,9000.0,2,NaN,NaN,NaN
22003,СТ-103-3708200,Якорь стартера СТ-103 Алтай (винтовой),NaN,NaN,3840.0,2,NaN,NaN,NaN


## Add Spare Part Unit (spu)

In [28]:
new_data = pd.read_excel('datasets/production.xlsx')

new_data = new_data.rename(columns={
    'Каталожный номер': 'vendor_code',
    'Наименование товара': 'title',
    'безнал': 'price',
    'Категория': 'category',
    'Подкатегория': 'spare_parts_unit'
})

new_data = new_data.drop(columns=['Цена наличка'])

categories = {
    'КамАЗ': 1,
    'МАЗ': 2,
    'ЗИЛ': 3,
    'Зил': 3,
    "УРАЛ": 4,
    'Урал': 4,
    "ПАЗ": 5,
    "ГАЗ": 6
}

spare_parts_units = {
    'Ходовая Часть': 1,
    'Механизмы управления': 2,
    'Двигатель': 3,
    'Дополнительное оборудование': 4,
    'Трансмиссия': 5
}


new_data['price'] = new_data['price'].astype({'price': float})

for cat in categories:
    new_data['category'] = new_data['category'].replace(cat, categories[cat], regex=True)

for unit in spare_parts_units:
    new_data['spare_parts_unit'] = new_data['spare_parts_unit'].replace(unit, spare_parts_units[unit], regex=True)

new_data

,vendor_code,title,price,category,spare_parts_unit
0,5511-2918068,Башмак балансира с аллюминевой втулкой,4800.0,1,1
1,5511-2918068,Башмак балансира с бронзовой втулкой,6100.0,1,1
2,5511-2918068,Башмак балансира с фторопластовой втулкой,5150.0,1,1
3,5320-3422010,Вал карданный рулевой,1550.0,1,2
4,4310-3422010,Вал карданный рулевой,1900.0,1,1
...,...,...,...,...,...
114,100-3519200,Энергоаккумулятор тип 24 новый,3100.0,1,2
115,100-3519200,Энергоаккумулятор тип 24 бордовый,3200.0,1,2
116,661-3519100,Энергоаккумулятор тип 20/20 (матрешка) с/с,2050.0,1,2
117,661-3519200,Энергоаккумулятор тип 24/20 (матрешка) с/с,2050.0,1,2


## Merge tables last_dt and new_dt

In [29]:
df1 = new_data.merge(df, on=['vendor_code'], how='left')

df1 = df1.drop(columns=['title_y', 'category_y', 'price_y'])
df1 = df1.rename(columns={
    'title_x': 'title',
    'price_x': 'price',
    'category_x': 'category'
})

df1

,vendor_code,title,price,category,spare_parts_unit,short_description,description,image_original,image_128,image_432
0,5511-2918068,Башмак балансира с аллюминевой втулкой,4800.0,1,1,NaN,NaN,NaN,NaN,NaN
1,5511-2918068,Башмак балансира с бронзовой втулкой,6100.0,1,1,NaN,NaN,NaN,NaN,NaN
2,5511-2918068,Башмак балансира с фторопластовой втулкой,5150.0,1,1,NaN,NaN,NaN,NaN,NaN
3,5320-3422010,Вал карданный рулевой,1550.0,1,2,Вал карданный рулевой КАМАЗ 5320,Вал карданный рулевой КАМАЗ 5320,original/запчасть-на-фотосессии.jpg,128/128__запчасть-на-фотосессии.jpg,432/432__запчасть-на-фотосессии.jpg
4,4310-3422010,Вал карданный рулевой,1900.0,1,1,Вал карданный рулевой КАМАЗ 4310,Вал карданный рулевой КАМАЗ 4310,original/запчасть-на-фотосессии.jpg,128/128__запчасть-на-фотосессии.jpg,432/432__запчасть-на-фотосессии.jpg
...,...,...,...,...,...,...,...,...,...,...
114,100-3519200,Энергоаккумулятор тип 24 новый,3100.0,1,2,&nbsp;Предназначен для привода в действие торм...,&nbsp;Предназначен для привода в действие торм...,original/IMG_20200227_092811-scaled.jpg,128/128__IMG_20200227_092811-scaled.jpg,432/432__IMG_20200227_092811-scaled.jpg
115,100-3519200,Энергоаккумулятор тип 24 бордовый,3200.0,1,2,&nbsp;Предназначен для привода в действие торм...,&nbsp;Предназначен для привода в действие торм...,original/IMG_20200227_092811-scaled.jpg,128/128__IMG_20200227_092811-scaled.jpg,432/432__IMG_20200227_092811-scaled.jpg
116,661-3519100,Энергоаккумулятор тип 20/20 (матрешка) с/с,2050.0,1,2,NaN,NaN,NaN,NaN,NaN
117,661-3519200,Энергоаккумулятор тип 24/20 (матрешка) с/с,2050.0,1,2,"Энергоаккумулятор КАМАЗ-43114,65140,65115 ново...","Энергоаккумулятор КАМАЗ-43114,65140,65115 ново...",original/Energoakkumulyatory-tip-24-20.jpg,128/128__Energoakkumulyatory-tip-24-20.jpg,432/432__Energoakkumulyatory-tip-24-20.jpg


## Concat tables dt and midakm

In [30]:
df2 = pd.concat([df1, df])

df2 = df2.drop_duplicates(subset=['vendor_code', 'title'])

df2 = df2.dropna(subset=['vendor_code'])
df2 = df2.dropna(subset=['title'])

df2 = df2.reset_index(drop=True)
df2['spare_parts_unit'] = df2['spare_parts_unit'] .fillna('')

df2

,vendor_code,title,price,category,spare_parts_unit,short_description,description,image_original,image_128,image_432
0,5511-2918068,Башмак балансира с аллюминевой втулкой,4800.0,1,1,NaN,NaN,NaN,NaN,NaN
1,5511-2918068,Башмак балансира с бронзовой втулкой,6100.0,1,1,NaN,NaN,NaN,NaN,NaN
2,5511-2918068,Башмак балансира с фторопластовой втулкой,5150.0,1,1,NaN,NaN,NaN,NaN,NaN
3,5320-3422010,Вал карданный рулевой,1550.0,1,2,Вал карданный рулевой КАМАЗ 5320,Вал карданный рулевой КАМАЗ 5320,original/запчасть-на-фотосессии.jpg,128/128__запчасть-на-фотосессии.jpg,432/432__запчасть-на-фотосессии.jpg
4,4310-3422010,Вал карданный рулевой,1900.0,1,1,Вал карданный рулевой КАМАЗ 4310,Вал карданный рулевой КАМАЗ 4310,original/запчасть-на-фотосессии.jpg,128/128__запчасть-на-фотосессии.jpg,432/432__запчасть-на-фотосессии.jpg
...,...,...,...,...,...,...,...,...,...,...
22112,840-1117030,Элемент ФТОТ 840 (Евро-3) (нитка) СЕДАН,231.6,2,,NaN,NaN,NaN,NaN,NaN
22113,2501-3708200-21,Якорь (для старт.2501.3708000-21) 10зуб. н/о П...,9000.0,2,,NaN,NaN,NaN,NaN,NaN
22114,2501-3708200-20(01),Якорь стартера МАЗ (11 зубьев) старого образца...,9000.0,2,,NaN,NaN,NaN,NaN,NaN
22115,СТ-103-3708200,Якорь стартера СТ-103 Алтай (винтовой),3840.0,2,,NaN,NaN,NaN,NaN,NaN


# Start classification

In [31]:
word_series = df2['title'].str.split(' ', expand=True).loc[:, 0:1]

word_series = word_series.loc[word_series[1] != '']
word_series = word_series.loc[word_series[0] != '']

two_words = word_series.loc[:, 0] + " " + word_series.loc[:, 1]

two_words

0             Башмак балансира
1             Башмак балансира
2             Башмак балансира
3                Вал карданный
4                Вал карданный
                 ...          
22112             Элемент ФТОТ
22113               Якорь (для
22114           Якорь стартера
22115           Якорь стартера
22116    Ящик инструментальный
Length: 21606, dtype: object

In [32]:
first_word_series = df2['title'].str.split(' ', expand=True).loc[:, 0]

df3 = pd.concat([df2, first_word_series], axis=1)
df3 = df3.rename(columns={0: 'first_word'})

df3 = pd.concat([df3, two_words], axis=1)
df3 = df3.rename(columns={0: 'two_words'})
df3

,vendor_code,title,price,category,spare_parts_unit,short_description,description,image_original,image_128,image_432,first_word,two_words
0,5511-2918068,Башмак балансира с аллюминевой втулкой,4800.0,1,1,NaN,NaN,NaN,NaN,NaN,Башмак,Башмак балансира
1,5511-2918068,Башмак балансира с бронзовой втулкой,6100.0,1,1,NaN,NaN,NaN,NaN,NaN,Башмак,Башмак балансира
2,5511-2918068,Башмак балансира с фторопластовой втулкой,5150.0,1,1,NaN,NaN,NaN,NaN,NaN,Башмак,Башмак балансира
3,5320-3422010,Вал карданный рулевой,1550.0,1,2,Вал карданный рулевой КАМАЗ 5320,Вал карданный рулевой КАМАЗ 5320,original/запчасть-на-фотосессии.jpg,128/128__запчасть-на-фотосессии.jpg,432/432__запчасть-на-фотосессии.jpg,Вал,Вал карданный
4,4310-3422010,Вал карданный рулевой,1900.0,1,1,Вал карданный рулевой КАМАЗ 4310,Вал карданный рулевой КАМАЗ 4310,original/запчасть-на-фотосессии.jpg,128/128__запчасть-на-фотосессии.jpg,432/432__запчасть-на-фотосессии.jpg,Вал,Вал карданный
...,...,...,...,...,...,...,...,...,...,...,...,...
22112,840-1117030,Элемент ФТОТ 840 (Евро-3) (нитка) СЕДАН,231.6,2,,NaN,NaN,NaN,NaN,NaN,Элемент,Элемент ФТОТ
22113,2501-3708200-21,Якорь (для старт.2501.3708000-21) 10зуб. н/о П...,9000.0,2,,NaN,NaN,NaN,NaN,NaN,Якорь,Якорь (для
22114,2501-3708200-20(01),Якорь стартера МАЗ (11 зубьев) старого образца...,9000.0,2,,NaN,NaN,NaN,NaN,NaN,Якорь,Якорь стартера
22115,СТ-103-3708200,Якорь стартера СТ-103 Алтай (винтовой),3840.0,2,,NaN,NaN,NaN,NaN,NaN,Якорь,Якорь стартера


In [33]:
word__spare_unit = {'Вал': 5, "Бак": 4, "Трубка": 6, "Кольцо": 6, "Прокладка": 6, "Болт": 7, "Шайба": 7, "Шестерня": 5, 'Гайка': 7}
two_words__spare_unit = {'Кронштейн крепления': 7}

for row_index in range(len(df3)):
    spare_parts_unit = df3.loc[row_index, 'spare_parts_unit']

    if spare_parts_unit:
        first_word = df3.loc[row_index, 'first_word']
        word__spare_unit[first_word] = spare_parts_unit
    else:
        break

word__spare_unit

{'Вал': 1,
 'Бак': 4,
 'Трубка': 6,
 'Кольцо': 6,
 'Прокладка': 6,
 'Болт': 7,
 'Шайба': 7,
 'Шестерня': 5,
 'Гайка': 7,
 'Башмак': 1,
 'Гидромуфта': 3,
 'Головка': 3,
 'ГУР': 2,
 'ГУР-6540': 2,
 'Диск': 5,
 'Камера': 2,
 'Картер': 5,
 'КОМ': 4,
 'Компрессор': 2,
 'Корзина': 5,
 'Кулиса': 5,
 'Механизм': 5,
 'МОД': 5,
 'Наконечник': 2,
 'Насос': 2,
 'ПГУ': 5,
 'Сошка': 2,
 'Стартер': 3,
 'Тяга': 2,
 'Штанга': 1,
 'Энергоаккумулятор': 2}

In [34]:
for row_index in range(len(df3)):
    for word in word__spare_unit:
        title = df3.loc[row_index, 'title']
        if word in title:
            df3.loc[row_index, 'spare_parts_unit'] = word__spare_unit[word]
            break
    else:
        for two_word in two_words__spare_unit:
            title = df3.loc[row_index, 'title']
            if two_word in title:
                df3.loc[row_index, 'spare_parts_unit'] = two_words__spare_unit[two_word]
                break
        else:
            df3.loc[row_index, 'spare_parts_unit'] = 0

df3.groupby('spare_parts_unit')['vendor_code'].count()

spare_parts_unit
0    16287
1      734
2      964
3      190
4      220
5      865
6     1646
7     1211
Name: vendor_code, dtype: int64

In [35]:
one_word = df3.loc[df3['spare_parts_unit'] == '', 'first_word'].value_counts()
one_word = one_word.loc[one_word > 100]

two_words_ = df3.loc[df3['spare_parts_unit'] == '', 'two_words'].value_counts()

two_words_

Series([], Name: two_words, dtype: int64)

In [36]:
df4 = df3.drop(columns=['first_word', 'two_words'])

df4['price'] = df4['price'].fillna(0)
df4['category'] = df4['category'].fillna(0)
df4 = df4.fillna(0)

df4

,vendor_code,title,price,category,spare_parts_unit,short_description,description,image_original,image_128,image_432
0,5511-2918068,Башмак балансира с аллюминевой втулкой,4800.0,1,1,0,0,0,0,0
1,5511-2918068,Башмак балансира с бронзовой втулкой,6100.0,1,1,0,0,0,0,0
2,5511-2918068,Башмак балансира с фторопластовой втулкой,5150.0,1,1,0,0,0,0,0
3,5320-3422010,Вал карданный рулевой,1550.0,1,1,Вал карданный рулевой КАМАЗ 5320,Вал карданный рулевой КАМАЗ 5320,original/запчасть-на-фотосессии.jpg,128/128__запчасть-на-фотосессии.jpg,432/432__запчасть-на-фотосессии.jpg
4,4310-3422010,Вал карданный рулевой,1900.0,1,1,Вал карданный рулевой КАМАЗ 4310,Вал карданный рулевой КАМАЗ 4310,original/запчасть-на-фотосессии.jpg,128/128__запчасть-на-фотосессии.jpg,432/432__запчасть-на-фотосессии.jpg
...,...,...,...,...,...,...,...,...,...,...
22112,840-1117030,Элемент ФТОТ 840 (Евро-3) (нитка) СЕДАН,231.6,2,0,0,0,0,0,0
22113,2501-3708200-21,Якорь (для старт.2501.3708000-21) 10зуб. н/о П...,9000.0,2,0,0,0,0,0,0
22114,2501-3708200-20(01),Якорь стартера МАЗ (11 зубьев) старого образца...,9000.0,2,0,0,0,0,0,0
22115,СТ-103-3708200,Якорь стартера СТ-103 Алтай (винтовой),3840.0,2,0,0,0,0,0,0


In [37]:
import psycopg2
import psycopg2.extras as extras


def execute_values(conn, df, table):

	tuples = [tuple(x) for x in df.to_numpy()]

	cols = ','.join(list(df.columns))
	# SQL query to execute
	query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
	cursor = conn.cursor()
	try:
		extras.execute_values(cursor, query, tuples)
		conn.commit()
	except (Exception, psycopg2.DatabaseError) as error:
		print("Error: %s" % error)
		conn.rollback()
		cursor.close()
		return 1
	print("the dataframe is inserted")
	cursor.close()


conn = psycopg2.connect(
	database="shop", user='postgres', password='vdnjvjnjhgDFGHJXCV56483754343', host='178.21.8.81', port='5432'
)

execute_values(conn, df4, 'products')

the dataframe is inserted
